<a href="https://colab.research.google.com/github/KoreaUnivseokjinmoon/seokjinmoonNLU/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_en(Friends).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Pretrained Model 설치

In [ ]:
# wget을 활용해서 bert 모델 다운로드 가능
import os
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip

if "bert" not in os.listdir():
  os.makedirs("bert")
else:
  pass

import zipfile
import shutil
         
bert_zip = zipfile.ZipFile('multi_cased_L-12_H-768_A-12.zip')
bert_zip.extractall('bert')
 
bert_zip.close()

def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

copytree("bert/multi_cased_L-12_H-768_A-12", "bert")

--2020-12-21 12:03:19--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.128, 172.217.204.128, 172.217.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip’

multi_cased_L-12_H- 100%[===================>] 632.19M   177MB/s    in 3.7s    

2020-12-21 12:03:23 (173 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip’ saved [662903077/662903077]



2. 필요한 Module Import

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

import pandas as pd
import numpy as np  
import re
import pickle

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

In [ ]:
import os

3. Keras-bert Import

In [ ]:
!pip install keras-bert

  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=c4485ccc3540942bf5f1390b65c9c531c40d848e906f0bfb0859e49c5151aec5
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for keras-transformer: filename=keras_transformer-0.38.0-cp36-none-any.whl size=12944 sha256=dae571776db86a0ecd12479072ac9e60825bd56ceb4a291e7eae5378352bfabd
  Stored in directory: /root/.cache/pip/wheels/e5/fb/3a/37b2b9326c799aa010ae46a04ddb04f320d8c77c0b7e837f4e
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7553 sha256=cc8e3a802c302922b3f4d30b0f81178848a040d7175447a3b09b74aa1810267d
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp36-none-any.whl size=15612 sha256=0c53b9d1aef27009d833eaa2c7ea2040512c525defaabb7df46073afa7a37755
  Stor

Keras-bert 라이브러리에서 필요한 Module Import

In [ ]:
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

4. Train, Test 데이터 다운

In [ ]:
!git clone https://github.com/KoreaUnivseokjinmoon/seokjinmoonNLU.git

Cloning into 'seokjinmoonNLU'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 25 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.


In [ ]:
os.listdir('seokjinmoonNLU')

['testSet', '.git', 'friends_en', 'README.md']

In [ ]:
import json

obj = {}
obj2 = {}

train_path = "seokjinmoonNLU/friends_en/"+"friends_train.json"
dev_path = "seokjinmoonNLU/friends_en/"+"friends_dev.json"
test_path = "seokjinmoonNLU/friends_en/"+"friends_test.json"


#train = json.loads(train_path)

#with open(train_path) as jsonfile:
#    obj = json.load(jsonfile)


#with open("./seokjinmoonNLU/friends_en/"+"friends_test_trans.json", 'w', encoding='utf-8') as f:
#    json.dump(obj, f, ensure_ascii=False)


#with open("./seokjinmoonNLU/friends_en/"+"friends_test_trans.json", encoding='utf-8') as jsonfile:
#    obj2 = json.load(jsonfile)


train = pd.read_json("seokjinmoonNLU/friends_en/"+"friends_train.json", encoding='utf-8-sig')
dev = pd.read_json("seokjinmoonNLU/friends_en/"+"friends_dev.json", encoding='utf-8-sig')
test = pd.read_json("seokjinmoonNLU/friends_en/"+"friends_test.json", encoding='utf-8-sig')

size = train.size

trainDF = pd.DataFrame(index=range(0,size), columns=['emotion', 'utterance'])
print(trainDF.dtypes)

trainDF = trainDF.fillna(0)
print(train[0][0]['emotion'])
#print(train[1][0])
for i in range(0, 719):
  for j in range(0, 23):
    trainDF.iloc[i+j, trainDF.columns.get_loc('emotion')] = train[i][j]['emotion']
    trainDF.iloc[i+j, trainDF.columns.get_loc('utterance')] = train[i][j]['utterance']

#trainDF.insert(loc=0, column='emotion', value=1)


print(trainDF)
#trainDF



#temp.columns = ['id', 'document']


emotion      object
utterance    object
dtype: object
neutral


TypeError: ignored